In [ ]:
# get all the packages

using Printf
using PyPlot
using Distributions

In [ ]:
L = 10.0
l = L / 2

In [ ]:
@inline function enforce_bounds!(xs::Array{Float64, 1})
    #xs[abs.(xs) .> L / 2] = sign.(xs[abs.(xs) .> L / 2]) .* L ./ 2
    for i in 1:length(xs)
        if xs[i] > l
            xs[i] = l
        elseif xs[i] < -l
            xs[i] = -l
        end
    end
end

In [ ]:
@time enforce_bounds!([-6.0, 6.0])

In [ ]:
@inline function box_muller(alpha::Float64)::Array{Float64, 1}
    r1 = rand()
    r2 = rand()
    
    ys = alpha .* [sqrt(-2 * log(r1 * sign(r1))) * cos(r2), sqrt(-2 * log(r1 * sign(r1))) * sin(r2)]
    return ys
end

In [ ]:
@time box_muller(2.75)

In [ ]:
function npc_simulation(D::Float64;
                        ntmax::Float64=1e6, dt::Float64=0.001, n_samples::Int64=2000)::Array{Float64, 1}
    # define array for storing the time it takes to reach the npc from the starting position
    time_capture = Array{Float64, 1}(undef, n_samples)
    
    # define the normal distribution for randomly perturbing
    alpha = sqrt(2 * D * dt)
    move_distr = Normal(0.0, alpha)
    
    # define the location of the npc for capture
    NPCLocation = [-l, 0]
    NPCSize = 0.01
    NPCSize_sqr = NPCSize ^ 2
    
    #define xs here so it doesn't need to reallocate
    xs = [-1.0, -1.0]
    
    # iterate through the number of samples to find the timing requirement
    for sample_num in 1:n_samples  
        xs .= [l, 0]
        # iterate through the time for the given sample
        for i in 1:ntmax
            # perturb the particle by a random amount based on our normal distribution
            #xs .+= box_muller(alpha)
            xs .+= vec(rand(move_distr, (2, 1)))
            
            # ensure that the particle stays within the bounds of the simulation
            enforce_bounds!(xs)
            
            # test for capture
            if (sum((xs .- NPCLocation) .^ 2) < NPCSize_sqr)
                time_capture[sample_num] = dt * i
                break
            end
        end
    end
    
    return time_capture
end

In [ ]:
@time npc_simulation(10.0; n_samples=1)
@time npc_simulation(10.0)